In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('hexoskin_thorax_training_set_10_final.xlsx')
df

,exp_diff_mean,exp_dur_sd,duty_cycle_mean,RRV_RMSSD,ie_ratio_mean,insp_flow_sd,insp_amp_sd,exp_amp_sd,ie_ratio_sd,exp_dur_mean,Task_Label,Participant,Classification
0,0.127790,0.156604,0.492097,376.386712,0.971761,1.445912e+06,5.297158e+06,5.304442e+06,0.074256,1.683594,12.0a,10785,0
1,0.201451,0.152211,0.486992,351.137516,0.950939,1.034691e+06,2.361021e+06,2.676608e+06,0.057593,1.749512,12.0b,10785,0
2,0.199219,0.213642,0.476697,457.608476,0.913378,1.890795e+06,3.910462e+06,3.564286e+06,0.067618,2.283203,12.0c,10785,0
3,0.285156,0.213173,0.477750,439.827676,0.918119,1.316481e+06,2.710758e+06,2.535948e+06,0.082441,1.953683,12.0d,10785,0
4,0.159375,0.213828,0.481671,457.645154,0.936229,9.634884e+05,3.059318e+06,3.092634e+06,0.115349,2.149088,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,0.203125,0.200555,0.490259,334.175635,0.967894,2.209139e+06,5.263487e+06,7.984288e+06,0.108475,1.498828,85,98586,0
2900,0.921387,0.725393,0.401725,1209.145284,0.933997,1.280213e+07,2.472549e+07,1.682432e+07,0.893126,1.939236,87,98586,1
2901,0.106337,0.117894,0.496388,185.358320,0.987026,2.990174e+06,5.322750e+06,5.857152e+06,0.051696,1.456641,89,98586,0
2902,2.145182,1.447558,0.412197,3053.223554,0.748645,1.441962e+07,1.907822e+07,1.914789e+07,0.283130,2.785156,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[1.27790200e-01, 1.56603600e-01, 4.92097400e-01, ...,
         5.30444166e+06, 7.42560000e-02, 1.68359380e+00],
        [2.01450900e-01, 1.52211400e-01, 4.86992000e-01, ...,
         2.67660774e+06, 5.75926000e-02, 1.74951170e+00],
        [1.99218800e-01, 2.13641900e-01, 4.76696800e-01, ...,
         3.56428630e+06, 6.76182000e-02, 2.28320310e+00],
        ...,
        [1.06336800e-01, 1.17894400e-01, 4.96388400e-01, ...,
         5.85715185e+06, 5.16958000e-02, 1.45664060e+00],
        [2.14518230e+00, 1.44755800e+00, 4.12197100e-01, ...,
         1.91478947e+07, 2.83129900e-01, 2.78515620e+00],
        [1.23535200e-01, 1.50456800e-01, 4.80522600e-01, ...,
         8.14725521e+06, 4.99940000e-02, 1.67187500e+00]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['exp_diff_mean',
  'exp_dur_sd',
  'duty_cycle_mean',
  'RRV_RMSSD',
  'ie_ratio_mean',
  'insp_flow_sd',
  'insp_amp_sd',
  'exp_amp_sd',
  'ie_ratio_sd',
  'exp_dur_mean'],
 'participants': arr

In [4]:
#running the nested cross-validation for the Accelerometer method using Random Forest
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

rfc = RandomForestClassifier(random_state=random_seed, class_weight="balanced")

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

nested_accuracy_scores:    [0.96969697 0.89393939 0.84848485 0.83333333 0.98484848 0.89393939
 0.98484848 0.83333333 0.92424242 0.89393939 0.90909091 0.74242424
 0.75757576 0.78787879 0.92424242 0.87878788 1.         0.8030303
 0.95454545 0.87878788 0.81818182 0.95454545 0.77272727 0.89393939
 0.87878788 0.98484848 0.93939394 0.98484848 0.71212121 0.74242424
 0.92424242 0.95454545 0.84848485 0.95454545 0.8030303  0.86363636
 0.92424242 0.89393939 0.96969697 0.93939394 0.8030303  0.8030303
 0.68181818 0.87878788]
mean score:            0.8778
nested_precision_scores:    [0.96969697 0.89466089 0.85885167 0.88571429 0.98520085 0.89337255
 0.98545455 0.87052342 0.92981093 0.91788856 0.91831852 0.82724683
 0.85454545 0.79265734 0.92398198 0.89747475 1.         0.80154426
 0.95959596 0.8885851  0.87878788 0.95538803 0.86013986 0.89525499
 0.90909091 0.98520085 0.94213287 0.98545455 0.79435379 0.73623129
 0.93730408 0.95959596 0.8790107  0.95959596 0.85479682 0.90082645
 0.92398198 0.89525499

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.9583
Specificity: 0.9762
Fold 2:
Sensitivity: 0.7917
Specificity: 0.9524
Fold 3:
Sensitivity: 0.8750
Specificity: 0.8333
Fold 4:
Sensitivity: 1.0000
Specificity: 0.7381
Fold 5:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 6:
Sensitivity: 0.8333
Specificity: 0.9286
Fold 7:
Sensitivity: 1.0000
Specificity: 0.9762
Fold 8:
Sensitivity: 0.9583
Specificity: 0.7619
Fold 9:
Sensitivity: 0.9583
Specificity: 0.9048
Fold 10:
Sensitivity: 1.0000
Specificity: 0.8333
Fold 11:
Sensitivity: 0.9583
Specificity: 0.8810
Fold 12:
Sensitivity: 0.9583
Specificity: 0.6190
Fold 13:
Sensitivity: 1.0000
Specificity: 0.6190
Fold 14:
Sensitivity: 0.7500
Specificity: 0.8095
Fold 15:
Sensitivity: 0.8750
Specificity: 0.9524
Fold 16:
Sensitivity: 0.9583
Specificity: 0.8333
Fold 17:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 18:
Sensitivity: 0.7083
Specificity: 0.8571
Fold 19:
Sensitivity: 1.0000
Specificity: 0.9286
Fold 20:
Sensitivity: 0.9167
Specificity: 0.8571
Fold 21:
Sensitivity: 1.0000


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.877754820936639, SD=0.0817827897816719, 95% CI=(0.8535899948469639, 0.9019196470263142), Min=0.6818181818181818, Max=1.0
test_precision: Mean=0.8999186563582607, SD=0.06126893341691901, 95% CI=(0.8818151760945985, 0.918022136621923), Min=0.7362312939020665, Max=1.0
test_recall: Mean=0.877754820936639, SD=0.0817827897816719, 95% CI=(0.8535899948469639, 0.9019196470263142), Min=0.6818181818181818, Max=1.0
test_f1_score: Mean=0.8730012752574791, SD=0.08342278266503601, 95% CI=(0.8483518711420286, 0.8976506793729295), Min=0.6817451205510907, Max=1.0
test_roc_auc: Mean=0.9549062049062048, SD=0.05281475451014045, 95% CI=(0.9393007289727774, 0.9705116808396322), Min=0.7514880952380952, Max=1.0
sensitivity_scores: Mean=0.931818181818182, SD=0.09211961257466697, 95% CI=(0.9045990759004482, 0.9590372877359158), Min=0.5416666666666666, Max=1.0
specificity_scores: Mean=0.8468614718614718, SD=0.12123110855729857, 95% CI=(0.8110406266351552, 0.8826823170877883), Min=0.523809523

In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')

Fold 1 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}
Fold 2 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}
Fold 3 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 4 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 5 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 6 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 7 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 8 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}
Fold 9 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: max_depth - Most frequent value: 6 (Count: 44)
Hyperparameter: max_features - Most frequent value: None (Count: 44)
Hyperparameter: max_leaf_nodes - Most frequent value: 9 (Count: 44)
Hyperparameter: n_estimators - Most frequent value: 50 (Count: 14)
